<a href="https://colab.research.google.com/github/arifmhassan/Spark-Python-Data-Ingestion/blob/master/RetailSalespythonRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This section course Highlight

In [319]:
# initialize spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz 
!tar xf spark-2.4.5-bin-hadoop2.7.tgz 
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()

# Regression

In [320]:
# Source  git branch clone

%%bash
#-b my-branch
git clone -b master https://github.com/databricks/Spark-The-Definitive-Guide.git master

fatal: destination path 'master' already exists and is not an empty directory.


In [321]:
ls master/data/retail-data/all

online-retail-dataset.csv


In [322]:
sales=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("master/data/retail-data/all/online-retail-dataset.csv")

In [323]:
sales.show(1,False)

+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+
only showing top 1 row



In [324]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sales

tokenizer = Tokenizer(inputCol="Description", outputCol="words")
wordsData = tokenizer.transform(sales)
wordsData.show(1,False)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
featurizedData.show(10,False)



+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------------------------------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |words                                   |
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------------------------------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|[white, hanging, heart, t-light, holder]|
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------------------------------------+
only showing top 1 row

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+------------------------------------------+--------------------------

In [325]:
simpleDF = spark.read.json("master/data/simple-ml")
simpleDF.show(2,False)

+-----+----+------+------------------+
|color|lab |value1|value2            |
+-----+----+------+------------------+
|green|good|1     |14.386294994851129|
|blue |bad |8     |14.386294994851129|
+-----+----+------+------------------+
only showing top 2 rows



In [326]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula="Country ~ . + Description:Country+StockCode")
supervised.fit(sales).transform(sales).show(2,False)

+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+-------------------------------------------------------------------------------------------------+-----+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |features                                                                                         |label|
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+-------------------------------------------------------------------------------------------------+-----+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|(217926,[14708,25899,29968,34190,48361,57450,57451,57452],[1.0,1.0,1.0,6.0,1.0,2.55,17850.0,1.0])|0.0  |
|536365   |71053    |WHITE METAL LANTERN               |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|

In [327]:
# SQL Transformer
from pyspark.ml.feature import SQLTransformer
basicTransformation = SQLTransformer()\
  .setStatement("""
    SELECT sum(Quantity) TotalQuantity, count(*) TotalCount ,count( StockCode), CustomerID
    FROM __THIS__
    GROUP BY CustomerID
  """)

basicTransformation.transform(sales).show()

+-------------+----------+----------------+----------+
|TotalQuantity|TotalCount|count(StockCode)|CustomerID|
+-------------+----------+----------------+----------+
|          265|        30|              30|     17420|
|          133|         8|               8|     16861|
|          567|        86|              86|     16503|
|         3065|       302|             302|     15727|
|         7430|       224|             224|     17389|
|           85|         9|               9|     15447|
|          241|        40|              40|     14450|
|          286|        86|              86|     13623|
|         2051|       187|             187|     13285|
|           23|        20|              20|     16339|
|           95|        29|              29|     14570|
|          224|        88|              88|     16386|
|          148|        22|              22|     18024|
|          406|       103|             103|     12940|
|          729|        30|              30|     17679|
|         

In [328]:
#Vector Assembler
from pyspark.ml.feature import VectorAssembler
va=VectorAssembler().setInputCols(["Quantity","UnitPrice","CustomerID"])
va.transform(sales).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|VectorAssembler_e2324295525e__output|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|                  [6.0,2.55,17850.0]|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|                  [6.0,3.39,17850.0]|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|                  [8.0,2.75,17850.0]|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|                  [6.0,3.39,17850.0]|
|   536365|   84029E

In [329]:
# continuous feature into bucket
from pyspark.ml.feature import Bucketizer
bucketBorders = [0,1, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("Quantity")
bucketer.transform(sales).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Bucketizer_27919e312464__output|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|                            2.0|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|                            2.0|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|                            2.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|                            2.0|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1

In [330]:
df = spark.read.load("master/data/regression")

In [331]:
df.show()

+--------------+-----+
|      features|label|
+--------------+-----+
|[3.0,10.1,3.0]|  2.0|
| [2.0,1.1,1.0]|  1.0|
|[1.0,0.1,-1.0]|  0.0|
|[1.0,0.1,-1.0]|  0.0|
| [2.0,4.1,1.0]|  2.0|
+--------------+-----+



# Regression

In [332]:
!curl  -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4179k    0 4179k    0     0  1217k      0 --:--:--  0:00:03 --:--:-- 1216k


In [333]:
import pandas as pd
counts = pd.read_csv('FremontBridge.csv',index_col='Date',parse_dates=True)
#weather = pd.read_csv('data/BicycleWeather.csv', index_col='DATE', parse_dates=True)
# rename columns
counts=counts.rename(columns={"Fremont Bridge Total":"DriveThru1","Fremont Bridge East Sidewalk":"Catering","Fremont Bridge West Sidewalk":"Online"})

#daily sum
daily = counts.resample('d').sum()
#weekly sum
weekly = counts.resample('W').sum()
# Monthly
montly = counts.resample('M').sum()



In [334]:
counts.head()

,DriveThru1,Catering,Online
Date,,,
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 02:00:00,2.0,1.0,1.0
2012-10-03 03:00:00,5.0,2.0,3.0
2012-10-03 04:00:00,7.0,6.0,1.0


In [335]:
daily.head()

,DriveThru1,Catering,Online
Date,,,
2012-10-03,7042.0,3520.0,3522.0
2012-10-04,6950.0,3416.0,3534.0
2012-10-05,6296.0,3116.0,3180.0
2012-10-06,4012.0,2160.0,1852.0
2012-10-07,4284.0,2382.0,1902.0


In [336]:
daily['total']=daily.sum(axis=1)

In [337]:
daily.head()

,DriveThru1,Catering,Online,total
Date,,,,
2012-10-03,7042.0,3520.0,3522.0,14084.0
2012-10-04,6950.0,3416.0,3534.0,13900.0
2012-10-05,6296.0,3116.0,3180.0,12592.0
2012-10-06,4012.0,2160.0,1852.0,8024.0
2012-10-07,4284.0,2382.0,1902.0,8568.0


In [338]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

In [339]:
daily.head()

,DriveThru1,Catering,Online,total,Mon,Tue,Wed,Thu,Fri,Sat,Sun
Date,,,,,,,,,,,
2012-10-03,7042.0,3520.0,3522.0,14084.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2012-10-04,6950.0,3416.0,3534.0,13900.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2012-10-05,6296.0,3116.0,3180.0,12592.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2012-10-06,4012.0,2160.0,1852.0,8024.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2012-10-07,4284.0,2382.0,1902.0,8568.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [340]:
# Drop any rows with null values
daily.dropna(axis=0, how='any', inplace=True)

column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
X = daily[column_names]


In [341]:
from sklearn.linear_model import LinearRegression
y = daily['total']

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
daily['predicted'] = model.predict(X)

In [342]:
daily.columns

Index(['DriveThru1', 'Catering', 'Online', 'total', 'Mon', 'Tue', 'Wed', 'Thu',
       'Fri', 'Sat', 'Sun', 'predicted'],
      dtype='object')

# Regression


In [343]:
netsales=pd.read_csv("https://raw.githubusercontent.com/arifmhassan/Spark-Python-Data-Ingestion/master/Data")

In [344]:
netsales=netsales.rename(columns={"Business Date":"Date","Location Name":"Location","Net Sales Total  (Sum)":"NetSales","Number of Guests (Sum)":"GuestCount"})


In [345]:
netsales=netsales.rename(columns={'Number of Guests  (Sum)':"GuestCount"})

In [346]:
netsales['Date']=pd.to_datetime(netsales['Date'])

netsales['Location']=netsales['Location'].astype('float')
netsales['GuestCount']=netsales['GuestCount'].astype('float')

In [ ]:
netsales=netsales.reset_index('Date')


In [ ]:
netsales.info()

In [ ]:
daily.info()

In [ ]:
#netsales=netsales.set_index('Date')
#netsales.head()
netsales.resample('d').sum()

In [ ]:
#netsales['Date']=pd.to_datetime(netsales['Date'])
netsales['GuestCount']=netsales['GuestCount'].astype('float')
netsales['Location']=netsales['Location'].astype('str')

In [ ]:
netsales.set_index('Date')
netsales.resample('d').sum()

In [ ]:
dailynetsales=netsales.groupby(["Date","Location"]).sum().reset_index()

In [ ]:
dailynetsales['WeekDay']=dailynetsales['Date'].dt.dayofweek
dailynetsales['Day']=dailynetsales['Date'].dt.day
dailynetsales['Week']=dailynetsales['Date'].dt.week
dailynetsales['Month']=dailynetsales['Date'].dt.month
dailynetsales['Year']=dailynetsales['Date'].dt.year

In [ ]:
dailynetsales.info()

In [ ]:
dailynetsales['NumberDate']=dailynetsales['Date'].astype('int')/100000000000

In [ ]:
dailynetsales.head()

In [ ]:
column_names = [ 'Location','NumberDate','Day','Week']
X = dailynetsales[column_names]
from sklearn.linear_model import LinearRegression
y = dailynetsales['GuestCount']

from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
dailynetsales['predicted'] = model.predict(X)




In [377]:
dailynetsales.head(20)

,Date,Location,GuestCount,WeekDay,Day,Week,Month,Year,predicted,NumberDate
0,2019-06-10,2.0,226.0,0,10,24,6,2019,260.625715,15601248.0
1,2019-06-11,2.0,294.0,1,11,24,6,2019,259.912951,15602112.0
2,2019-06-12,2.0,179.0,2,12,24,6,2019,259.200186,15602976.0
3,2019-06-13,2.0,284.0,3,13,24,6,2019,258.487421,15603840.0
4,2019-06-14,2.0,375.0,4,14,24,6,2019,257.774656,15604704.0
5,2019-06-15,2.0,302.0,5,15,24,6,2019,257.061892,15605568.0
6,2019-06-16,2.0,248.0,6,16,24,6,2019,256.349127,15606432.0
7,2019-06-17,2.0,166.0,0,17,25,6,2019,256.554453,15607296.0
8,2019-06-18,2.0,201.0,1,18,25,6,2019,255.841688,15608160.0
9,2019-06-19,2.0,243.0,2,19,25,6,2019,255.128923,15609024.0


# Spark Regression

In [348]:
# regression label and feature feature enginering using R language RF formula
from pyspark.ml.feature import RFormula

# suppose below is my dataframe 
dataset = spark.createDataFrame(
    [(7, "TX", 18, 1.0),
     (8, "PA", 12, 0.0),
     (9, "NY", 15, 0.0)],
    ["Store", "State", "Quantity", "OrderType"])

# feature extraction Quatity is label
formula = RFormula(
    formula="Quantity ~ Store + State",
    featuresCol="features",
    labelCol="label")

output = formula.fit(dataset).transform(dataset)
output.select("features", "label").show()

+-------------+-----+
|     features|label|
+-------------+-----+
|[7.0,1.0,0.0]| 18.0|
|[8.0,0.0,0.0]| 12.0|
|[9.0,0.0,1.0]| 15.0|
+-------------+-----+



In [349]:
# ML 
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel=lr.fit(output)

In [350]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print(lrModel.summary.totalIterations)
print(lrModel.summary.residuals.show())

Coefficients: [0.0,4.793987681223805,1.8441350568654684]
Intercept: 12.787292420636911
11
+-------------------+
|          residuals|
+-------------------+
|0.41871989813928323|
| -0.787292420636911|
| 0.3685725224976206|
+-------------------+

None


In [351]:
# my prediction
lrModel.summary.predictions.show()

+-----+-----+--------+---------+-------------+-----+------------------+
|Store|State|Quantity|OrderType|     features|label|        prediction|
+-----+-----+--------+---------+-------------+-----+------------------+
|    7|   TX|      18|      1.0|[7.0,1.0,0.0]| 18.0|17.581280101860717|
|    8|   PA|      12|      0.0|[8.0,0.0,0.0]| 12.0|12.787292420636911|
|    9|   NY|      15|      0.0|[9.0,0.0,1.0]| 15.0| 14.63142747750238|
+-----+-----+--------+---------+-------------+-----+------------------+



In [ ]:
#scratch
formula = RFormula(
    formula="Quantity ~ Store + State",
    featuresCol="features",
    labelCol="label")

output = formula.fit(dataset).transform(dataset)
output.select("features", "label").show()

In [364]:
salesdf=spark.createDataFrame( dailynetsales[['Date','Location','Day','Week','Year','GuestCount']])

In [372]:
salesdf.select("Location").distinct().show()

+--------+
|Location|
+--------+
|     2.0|
+--------+



In [ ]:
salesdf.show(1,False)
#feature engineering
formula = RFormula(
    formula="GuestCount ~  Day+Week+Year",
    featuresCol="features",
    labelCol="label")

output = formula.fit(salesdf).transform(salesdf)
output.select("features", "label").show()

In [376]:
# ML 
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel=lr.fit(output)
# my prediction
lrModel.summary.predictions.show().sort(Descending=True)

+-------------------+--------+---+----+----+----------+------------------+-----+------------------+
|               Date|Location|Day|Week|Year|GuestCount|          features|label|        prediction|
+-------------------+--------+---+----+----+----------+------------------+-----+------------------+
|2019-06-10 00:00:00|     2.0| 10|  24|2019|     226.0|[10.0,24.0,2019.0]|226.0| 254.6847596147458|
|2019-06-11 00:00:00|     2.0| 11|  24|2019|     294.0|[11.0,24.0,2019.0]|294.0|254.08995092162905|
|2019-06-12 00:00:00|     2.0| 12|  24|2019|     179.0|[12.0,24.0,2019.0]|179.0| 253.4951422285123|
|2019-06-13 00:00:00|     2.0| 13|  24|2019|     284.0|[13.0,24.0,2019.0]|284.0|252.90033353539556|
|2019-06-14 00:00:00|     2.0| 14|  24|2019|     375.0|[14.0,24.0,2019.0]|375.0|252.30552484227883|
|2019-06-15 00:00:00|     2.0| 15|  24|2019|     302.0|[15.0,24.0,2019.0]|302.0|251.71071614916207|
|2019-06-16 00:00:00|     2.0| 16|  24|2019|     248.0|[16.0,24.0,2019.0]|248.0|251.11590745604533|


AttributeError: ignored

In [ ]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print(lrModel.summary.totalIterations)
print(lrModel.summary.residuals.show())